In [ ]:
import pandas as pd
import pickle
import numpy as np
import re
from sklearn import linear_model


import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.ensemble import RandomForestRegressor
from sklearn import linear_model
from sklearn import ensemble 
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import BaggingRegressor

from lightgbm import LGBMRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import KFold
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error, mean_absolute_percentage_error
from tqdm import tqdm_notebook

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score, KFold

from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
from sklearn.linear_model import ElasticNet
from sklearn.kernel_ridge import KernelRidge
from sklearn.ensemble import RandomForestRegressor
from sklearn import preprocessing

from catboost import CatBoostRegressor


import joblib

import os
import warnings
warnings.filterwarnings('ignore') 

def get_score(y_test, y_pred):
    mae = mean_absolute_error(y_test,y_pred)
    mse = mean_squared_error(y_test,y_pred)
    rmse = np.sqrt(mse)
    r2 = r2_score(y_test,y_pred)
    
    return(mae, mse, rmse, r2)

In [ ]:
# 데이터 불러오기
#data = pd.read_csv('2008~2023_data.csv', engine='python')
data = pd.read_parquet('2008~2023_data.parquet')

In [ ]:
data.head()

In [ ]:
data.info()

In [ ]:
data = data[(data['year'] >= 2018) & (data['year'] <= 2023)]

In [ ]:
data['rainy'] = data['Rainfall_amt']>0.0
data['rainy'] = data['rainy'].astype(np.int32)

In [ ]:
data['log_get_all'] = np.log1p(data['get_all'])

## 푸리에특징을 통한 시간연속성 표현(HOUR)
- 간단한 푸리에 변환을 활용하여, hour의 시간연속성을 데이터에 표현하였습니다.

In [ ]:
def cyclical_encoding(x, max_val):
    sin_val = np.sin(2 * np.pi * x / max_val)
    cos_val = np.cos(2 * np.pi * x / max_val)
    return sin_val, cos_val

# hour 변수를 Cyclical Encoding으로 변환하여 대체하기
max_hour = 24
data['hour_sin'], data['hour_cos']= cyclical_encoding(data['hour'], max_hour)
data.drop('hour', axis=1, inplace=True)

# 결과 확인
data.head()

# 원핫인코딩 진행

In [ ]:
# 역번호 원핫인코딩을 위한 데이터 프레임 만들기
sub_num_data = pd.get_dummies(data['Station_num'], prefix='sub_num', prefix_sep='_')

# 호선 원핫인코딩을 위한 데이터 프레임 만들기
line_num_data = pd.get_dummies(data['Line_num'], prefix='line_num', prefix_sep='_')

# 년도 원핫인코딩을 위한 데이터 프레임 만들기
# year_data = pd.get_dummies(data['year'], prefix='year', prefix_sep='_')

# 요일별 원핫인코딩을 위한 데이터 프레임 만들기
weekday_data = pd.get_dummies(data['weekday'], prefix='weekday', prefix_sep='_')

# 월별 원핫인코딩을 위한 데이터 프레임 만들기
month_data = pd.get_dummies(data['month'], prefix='month', prefix_sep='_')



In [ ]:
selected_columns = ['log_get_all','holiday','rainy','hour_sin','hour_cos','year']
data = pd.concat([data[selected_columns],  sub_num_data, line_num_data,
                        weekday_data, month_data], axis=1)

In [ ]:
del sub_num_data, line_num_data, weekday_data, month_data

In [ ]:
dict_ = {}
for i in data.columns:
    dict_[i] = 0

print(dict_)


In [ ]:
data.head()

In [ ]:
# 독립변수, 종속변수 설정 하기 
X = data.drop(['log_get_all'],axis=1)
Y = data['log_get_all']


In [ ]:
data.head()

# catboost 모델링

In [ ]:
from sklearn.model_selection import TimeSeriesSplit, cross_val_score
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from catboost import CatBoostRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
import numpy as np

# Specify the start and end years for training and testing
train_start_year = 2018
train_end_year = 2021
test_start_year = 2022
test_end_year = 2023

# Filter the data based on the years
train_data = data[(data['year'] >= train_start_year) & (data['year'] <= train_end_year)]
test_data = data[(data['year'] >= test_start_year) & (data['year'] <= test_end_year)]

# Separate the features (X) and target variable (Y)
X_train = train_data.drop(['log_get_all'], axis=1)
Y_train = train_data['log_get_all']
X_test = test_data.drop(['log_get_all'], axis=1)
Y_test = test_data['log_get_all']

# Create a TimeSeriesSplit object for time series splitting
tscv = TimeSeriesSplit(n_splits=5)

# Variable initialization
log_mae_scores = []
mae_scores = []

# Create a list of models
models = [
    CatBoostRegressor(depth=12, iterations=1000, learning_rate=0.03, verbose=1, random_state=42),
]

# Outer loop: iterate over the time series segmentation
for train_index, test_index in tscv.split(X_train):
    X_train_fold, X_val_fold = X_train.iloc[train_index], X_train.iloc[test_index]
    y_train_fold, y_val_fold = Y_train.iloc[train_index], Y_train.iloc[test_index]
    
    # Inner loop: estimate the model's performance through cross-validation
    for model in models:
        model.fit(X_train_fold, y_train_fold)  # Train the model
        
        scores = cross_val_score(model, X_train_fold, y_train_fold, scoring='neg_mean_absolute_error', cv=5)
        log_mean_score = -scores.mean()  # Negate the scores since cross_val_score returns negative values
        log_mae_scores.append(log_mean_score)
        
        # Evaluate the model on the validation set
        y_pred = np.expm1(model.predict(X_val_fold))
        mae = mean_absolute_error(np.expm1(y_val_fold), y_pred)
        mae_scores.append(mae)
        
        # Save the trained model using joblib
        joblib.dump(model, f'catboost_model_0623_ver1.pkl')

        print(f'{model.__class__.__name__} :')
        print('Log MAE:', log_mean_score)
        print('MAE:', mae)
        print('=' * 50)

# Visualize the results
model_names = [model.__class__.__name__ for model in models]

plt.figure(figsize=(10, 6))
plt.bar(model_names, log_mae_scores, label='Log MAE')
plt.bar(model_names, mae_scores, label='MAE')
plt.xlabel('Model')
plt.ylabel('Mean Absolute Error (MAE)')
plt.title('Comparison of Mean Absolute Error (MAE)')
plt.xticks(rotation=45)
plt.legend()
plt.show()


In [ ]:
# Train the CatBoostRegressor model
catboost_model = CatBoostRegressor(depth=12, iterations=1000, learning_rate=0.03, verbose=1, random_state=42)
catboost_model.fit(X_train, Y_train)

# Get feature importances
feature_importances = catboost_model.get_feature_importance()

# Get original feature names (without one-hot encoding)
original_feature_names = [col.split('_')[0] for col in X_train.columns]

# Create a dictionary to store the combined feature importances
combined_feature_importances = {}

# Iterate over the feature importances and sum them for the original feature names
for feature_name, importance in zip(X_train.columns, feature_importances):
    original_feature_name = feature_name.split('_')[0]
    if original_feature_name in combined_feature_importances:
        combined_feature_importances[original_feature_name] += importance
    else:
        combined_feature_importances[original_feature_name] = importance

# Convert the combined feature importances dictionary to a DataFrame
importance_df = pd.DataFrame.from_dict(combined_feature_importances, orient='index', columns=['Importance'])
importance_df = importance_df.sort_values('Importance', ascending=False)

# Plot feature importances
plt.figure(figsize=(10, 6))
plt.barh(range(len(importance_df)), importance_df['Importance'], align='center')
plt.yticks(range(len(importance_df)), importance_df.index)
plt.xlabel('Feature Importance')
plt.ylabel('Feature')
plt.title('Combined Feature Importances (One-Hot Encoded Variables)')
plt.show()